# Создание базы данных

In [2]:
import sqlite3

In [21]:
# create_queries = ['''
# CREATE TABLE "texts" ( 
# 	"id"	INTEGER,
# 	"name"	TEXT UNIQUE,
# 	PRIMARY KEY("id" AUTOINCREMENT)
# );''', 
# '''
# CREATE TABLE "sents" (
# 	"id"	INTEGER,
# 	"text_id"	INTEGER,
# 	"sent"	TEXT,
# 	"lemmatized"	TEXT,
# 	"pos_in_text"	INTEGER,
# 	FOREIGN KEY("text_id") REFERENCES "texts"("id"),
# 	PRIMARY KEY("id" AUTOINCREMENT)
# );''', 

# '''
# CREATE TABLE "govern_models" (
# 	"id"	INTEGER,
# 	"model"	TEXT UNIQUE,
# 	PRIMARY KEY("id" AUTOINCREMENT)
# );
# ''',

# '''
# CREATE TABLE "math_imgs" (
# 	"id"	INTEGER,
# 	"math_tagname"	TEXT,
# 	"filename"	TEXT,
# 	PRIMARY KEY("id" AUTOINCREMENT)
# );
# ''',

# '''
# CREATE TABLE "math_tags" (
# 	"id"	INTEGER,
# 	"feature"	TEXT,
# 	PRIMARY KEY("id" AUTOINCREMENT)
# );
# ''',

# '''
# CREATE TABLE "lemmas" (
# 	"id"	INTEGER,
# 	"name"	TEXT UNIQUE,
# 	PRIMARY KEY("id" AUTOINCREMENT)
# );
# ''',

# '''
# CREATE TABLE "pos" (
# 	"id"	INTEGER,
# 	"name"	TEXT,
# 	PRIMARY KEY("id" AUTOINCREMENT)
# );
# ''',

# '''
# CREATE TABLE "deprels" (
# 	"id"	INTEGER,
# 	"name"	INTEGER,
# 	PRIMARY KEY("id" AUTOINCREMENT)
# );
# ''',

# '''
# CREATE TABLE "tokens" (
# 	"id"	INTEGER,
# 	"sent_id"	INTEGER,
# 	"word_in_sent"	INTEGER,
# 	"token"	TEXT,
# 	FOREIGN KEY("sent_id") REFERENCES "sents"("id") ON DELETE RESTRICT,
# 	PRIMARY KEY("id" AUTOINCREMENT)
# );
# ''', 

# '''
# CREATE TABLE "math_annotation" (
# 	"id"	INTEGER,
# 	"id_token_start"	INTEGER,
# 	"id_token_end"	INTEGER,
# 	"feature_id"	INTEGER,
# 	"govern_model_id"	INTEGER,
# 	FOREIGN KEY("feature_id") REFERENCES "math_tags"("id"),
# 	FOREIGN KEY("govern_model_id") REFERENCES "govern_models"("id"),
# 	FOREIGN KEY("id_token_start") REFERENCES "tokens"("id"),
# 	FOREIGN KEY("id_token_end") REFERENCES "tokens"("id"),
# 	PRIMARY KEY("id" AUTOINCREMENT)
# );
# ''',

# '''
# CREATE TABLE "grammar_annotation" (
# 	"token_id"	INTEGER,
# 	"lemma_id"	INTEGER,
# 	"pos_id"	INTEGER,
# 	"deprel_id"	INTEGER,
# 	"head_id"	INTEGER,
# 	FOREIGN KEY("lemma_id") REFERENCES "lemmas"("id"),
# 	FOREIGN KEY("token_id") REFERENCES "tokens"("id"),
# 	FOREIGN KEY("deprel_id") REFERENCES "deprels"("id"),
# 	FOREIGN KEY("head_id") REFERENCES "tokens"("id"),
# 	FOREIGN KEY("pos_id") REFERENCES "pos"("id"),
# 	PRIMARY KEY("token_id")
# );
# '''
#                  ]

In [23]:
# db_path = 'math_corpus_database.db'
# conn = sqlite3.connect(db_path, check_same_thread=False)
# cur = conn.cursor()

In [103]:
# for q in create_queries:
#     cur.execute(q)
#     conn.commit()
# conn.close()

# Парсинг и заполнение БД

In [3]:
!pip install spacy

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
!python -m spacy download ru_core_news_sm

Defaulting to user installation because normal site-packages is not writeable


[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



     ---------------------------------------- 15.3/15.3 MB 9.8 MB/s eta 0:00:00
[+] Download and installation successful
You can now load the package via spacy.load('ru_core_news_sm')


In [154]:
pip install tqdm

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
import spacy
nlp = spacy.load("ru_core_news_sm")

In [10]:
test_text = '''<?xml version='1.0' encoding='utf-8'?>
<document>
<header>
<textfile>Texts/limit_test.txt</textfile>
<lang>english</lang></header><body>
<segment id='45' features='math_tags' state='active' subcorp='' filename=''>И дальше мы говорим, что вот какое-то число <segment id='164' features='math_tags;term' state='active' subcorp='' filename=''><segment id='163' features='math_tags;term;var' state='active' subcorp='' filename=''>а</segment> большое</segment> является <segment id='165' features='math_tags;function' state='active' subcorp='' filename=''><segment id='166' features='math_tags;term' state='active' subcorp='' filename=''>пределом </segment><segment id='167' features='math_tags;visual' state='active' subcorp='' filename=''>при <segment id='168' features='math_tags;term' state='active' subcorp='' filename=''>икс </segment></segment><segment id='169' features='math_tags;visual' state='active' subcorp='' filename=''>стремящемся к <segment id='170' features='math_tags;term' state='active' subcorp='' filename=''><segment id='171' features='math_tags;term;var' state='active' subcorp='' filename=''>а</segment> маленькому</segment></segment></segment>, если, какое бы малое <segment id='173' features='math_tags;term' state='active' subcorp='' filename=''>эпсилон</segment> вы вот тут не взяли… Да, потому что <segment id='174' features='math_tags;term' state='active' subcorp='' filename=''>эпсилон</segment>, оно про <segment id='175' features='math_tags;term' state='active' subcorp='' filename=''>а большое</segment>.</segment>
<segment id='46' features='math_tags' state='active' subcorp='' filename=''>Какое бы маленькое <segment id='172' features='math_tags;term' state='active' subcorp='' filename=''>эпсилон</segment> будут не взяли, существует такое <segment id='176' features='math_tags;term' state='active' subcorp='' filename=''>дельта</segment>, что все <segment id='177' features='math_tags;term' state='active' subcorp='' filename=''>иксы</segment>, которые лежат вот здесь, попадают вот сюда.</segment>
</body>
</document>'''

In [6]:
# функция для красивого печатания
def bolded_print(sent_string, seg_span):
    start, end = seg_span
    print(sent_string[:start] + '\033[1m' + sent_string[start:end] + '\033[0m' + sent_string[end:])

In [105]:
# # функция для красивого печатания
# for seg_id in segments:
#     start, end = segments[seg_id].char_start, segments[seg_id].char_end
#     sent = segments[seg_id].sentence
#     print(seg_id)
#     print(sent[:start] + '\033[1m' + sent[start:end] + '\033[0m' + sent[end:])
#     print()

45
И дальше мы говорим, что вот какое-то число а большое является пределом при икс стремящемся к а маленькому, если, какое бы малое эпсилон вы вот тут не взяли…

164
И дальше мы говорим, что вот какое-то число а большое является пределом при икс стремящемся к а маленькому, если, какое бы малое эпсилон вы вот тут не взяли…

163
И дальше мы говорим, что вот какое-то число а большое является пределом при икс стремящемся к а маленькому, если, какое бы малое эпсилон вы вот тут не взяли…

165
И дальше мы говорим, что вот какое-то число а большое является пределом при икс стремящемся к а маленькому, если, какое бы малое эпсилон вы вот тут не взяли…

166
И дальше мы говорим, что вот какое-то число а большое является пределом при икс стремящемся к а маленькому, если, какое бы малое эпсилон вы вот тут не взяли…

167
И дальше мы говорим, что вот какое-то число а большое является пределом при икс стремящемся к а маленькому, если, какое бы малое эпсилон вы вот тут не взяли…

168
И дальше мы говорим

In [7]:
from collections import defaultdict

In [8]:
import re
from bs4 import BeautifulSoup

In [9]:
from tqdm import tqdm

In [10]:
from dataclasses import dataclass

@dataclass
class SegmentInfo:
    seg_text: str
    available_search_area: str
    parent_id: str
    prev_char_count: int = 0
    char_start: int = 0
    char_end: int = 0
    features: str = None
    govern_model: str = None
    sentence: str = None

In [21]:
class DBHandler:
    conn = None
    cur = None

    def __init__(self, db_path):
        self.conn = sqlite3.connect(db_path, check_same_thread=False)
        self.cur = self.conn.cursor()

    def __del__(self):
        self.conn.close()

    def add_text(self, text_name):
        
        self.cur.execute('''INSERT or IGNORE INTO texts (name)
                            VALUES (?)
                            RETURNING id''', (text_name,))
    
        added_id = self.cur.fetchone()
        self.conn.commit()
        
        if not added_id:
            self.cur.execute('''SELECT id
                                FROM texts
                                WHERE name = (?)''', (text_name, ))
            added_id = self.cur.fetchone()
        return added_id[0]
        
    def add_sent(self, text_id, sent, lemmatized, sent_count):
        
        self.cur.execute('''INSERT INTO sents (text_id, sent, lemmatized, pos_in_text)
                            VALUES (?, ?, ?, ?)
                            RETURNING id''', (text_id, sent, lemmatized, sent_count))
        
        added_id = self.cur.fetchone()
        self.conn.commit()
        return added_id[0]
    
    
    def add_tokens(self, tokens_info): 
        ":param tokens_info: list of tuples (sent_id, word_in_sent, token, char_start, char_end)"
        
        self.cur.executemany('''INSERT INTO tokens (sent_id, word_in_sent, token, char_start, char_end)
                                VALUES (?, ?, ?, ?, ?)''', tokens_info)
        
        self.conn.commit()
        
        self.cur.execute('''SELECT id 
                            FROM tokens
                            WHERE sent_id = (?)
                            ORDER BY word_in_sent''', (tokens_info[0][0],))
        added_ids = self.cur.fetchall()
        return [token_id[0] for token_id in added_ids]
    
    
    def add_lemmas(self, lemmas):
        lemmas = [(el,) for el in lemmas]
        self.cur.executemany('''INSERT or IGNORE
                            INTO lemmas (name)
                            VALUES (?)''', lemmas)
        self.conn.commit()
        
        lemma_ids = []
        for lemma in lemmas:
            self.cur.execute('''SELECT id
                                FROM lemmas
                                WHERE name = ?''', lemma)
            lemma_ids.append(self.cur.fetchone()[0])
        return lemma_ids
    
    
    def add_grammar_annot(self, token_grammar_info):
        ":param token_grammar_info: [(token_id, pos, deprel, head_id, lemma)]"
        
        self.cur.executemany('''INSERT INTO grammar_annotation (token_id, lemma_id, pos_id, deprel_id, head_id)
                                SELECT ?, 
                                        lemmas.id, 
                                        (SELECT pos.id FROM pos WHERE pos.name = ?),
                                        (SELECT deprels.id FROM deprels WHERE deprels.name = ?),
                                        ?
                                FROM lemmas
                                WHERE lemmas.name = ?''', token_grammar_info)
        self.conn.commit()
        
    def add_math_tags(self, tags):
        tags = [(el,) for el in tags]
        self.cur.executemany('''INSERT or IGNORE
                                INTO math_tags (feature)
                                VALUES (?)''', tags)
        self.conn.commit()
        
    def add_govern_models(self, models):
        models = [(el,) for el in models]
        self.cur.executemany('''INSERT or IGNORE
                                INTO govern_models (model)
                                VALUES (?)''', models)
        self.conn.commit()
        
    def get_sent_id(self, sentence):
        self.cur.execute('''SELECT id
                            FROM sents
                            WHERE sents.sent = (?)''', (sentence,))
#         print(sentence)
        return self.cur.fetchone()[0]
    
    
    def add_math_annotation(self, annotation):
        """:param annotation: [{
            'sentence': str, 
            'segment_text': str,
            'char_start': int, 
            'char_end': int,
            'annot': str, 
            'govern_model': str}
            }]"""
        
        annot = list(set([an['annot'] for an in annotation]))
        self.add_math_tags(annot)
        govern_models = list(set([an['govern_model'] for an in annotation if an['govern_model']]))
        self.add_govern_models(govern_models)
        
        sent_ids = {}
        for annot in annotation:
            if not sent_ids.get(annot['sentence']):
                sent_ids[annot['sentence']] = self.get_sent_id(annot['sentence'])
            annot['sentence_id'] = sent_ids[annot['sentence']]
            
        self.cur.executemany('''INSERT INTO math_annotation (feature_id, govern_model_id, id_token_start, id_token_end)
                                SELECT math_tags.id, 
                                (SELECT govern_models.id FROM govern_models WHERE govern_models.model = :govern_model),
                                (SELECT tokens.id FROM tokens 
                                WHERE tokens.char_start >= :char_start AND tokens.char_end <= :char_end AND tokens.sent_id = :sentence_id
                                ORDER BY tokens.word_in_sent 
                                LIMIT 1),
                                (SELECT tokens.id FROM tokens 
                                WHERE tokens.char_start >= :char_start AND tokens.char_end <= :char_end AND tokens.sent_id = :sentence_id
                                ORDER BY tokens.word_in_sent DESC
                                LIMIT 1)
                            FROM math_tags
                            WHERE math_tags.feature = :annot''', annotation)
        self.conn.commit()
        
    def change_text_status(self, text_id, new_status):
        self.cur.execute('''UPDATE texts
                            SET status_id = (?)
                            WHERE id = (?)''', (new_status, text_id))
        self.conn.commit()

In [13]:
class XML2Database:
    
    def __init__(self, xml_file, database_path, textname=None):
        
        with open(xml_filename, 'r', encoding='utf-8') as f:
            xml_file = f.read()
        self.bs_data = BeautifulSoup(xml_file, "xml")
        self.db = DBHandler(db_path)
        self.nlp = spacy.load("ru_core_news_sm")
        
        if not textname:
            textname = re.search(r'Texts\/(.*?)\.txt', 
                                 self.bs_data.find('textfile').get_text()).group(1)
        self.text_id = self.db.add_text(textname)
        
        
    @staticmethod
    def parse_sentence_info_(analysed_sent):
        parsed = {
            'tokens': [], 
            'lemmas': [], 
            'poses': [], 
            'deprels': [], 
            'head_id_in_sent': [], 
            'full_sentence': analysed_sent.text,
        }
        first_word_in_sent = 0

        for t in analysed_sent:
            parsed['tokens'].append(t.text)
            parsed['lemmas'].append(t.lemma_)
            parsed['poses'].append(t.pos_)
            parsed['deprels'].append(t.dep_.lower())

            if t.is_sent_start:
                first_word_in_sent = t.i

            parsed['head_id_in_sent'].append(t.head.i - first_word_in_sent)
        parsed['lemmatized_sent'] = ' '.join(parsed['lemmas'])
        return parsed
    
    @staticmethod
    def accum_token_info_(parsed, sent_id):
        n_tokens = len(parsed['tokens'])
        token_info = []
        sent_string = parsed['full_sentence']
        prev_chars_count = 0
        for i in range(n_tokens):
            token = parsed['tokens'][i]
            m = re.search(re.escape(token), sent_string)
            char_start, char_end = m.start(), m.end()

            token_info.append((sent_id, i, parsed['tokens'][i], prev_chars_count + char_start, prev_chars_count + char_end))
            prev_chars_count += char_end
            sent_string = sent_string[char_end:]  # обрезаем строку, чтобы короткие слова не искались в начале строки
        return token_info
    
    @staticmethod
    def accum_grammar_info_(parsed, added_tokens):
        n_tokens = len(parsed['tokens'])
        head_ids = [added_tokens[hi] for hi in parsed['head_id_in_sent']]
        token_grammar_info = [
        (
            added_tokens[i], 
            parsed['poses'][i],
            parsed['deprels'][i], 
            head_ids[i],
            parsed['lemmas'][i]
        ) for i in range(n_tokens)
        ]
        return token_grammar_info
        
    def extract_sentences(self):
        full_text = self.bs_data.find('body').get_text().strip()
        sent_count = 0
        for sent in tqdm(self.nlp(full_text).sents):
            sent_count += 1
            parsed_sentence = XML2Database.parse_sentence_info_(sent)
            
            lemmas = parsed_sentence['lemmas']
            added_lemmas = self.db.add_lemmas(lemmas)
            
            sent_text = parsed_sentence['full_sentence']
            sent_lemmatized = parsed_sentence['lemmatized_sent']
            sent_id = self.db.add_sent(self.text_id, sent_text, sent_lemmatized, sent_count)
            
            tokens_info = XML2Database.accum_token_info_(parsed_sentence, sent_id)
            added_tokens = self.db.add_tokens(tokens_info)
            
            grammar_info = XML2Database.accum_grammar_info_(parsed_sentence, added_tokens)
            self.db.add_grammar_annot(grammar_info)
        self.db.change_text_status(self.text_id, 2)
#         print('all sentences added to dabase')
            
    @staticmethod
    def parse_annotation_segments_(bs_data):
        segments = {}
        for s in bs_data.find('body').find_all('segment', recursive=True):
            parent_id = s.parent.get('id')
            segments[s.get('id')] = SegmentInfo(s.get_text().strip(), s.get_text().strip(), parent_id)
        for seg_id in segments:
            segment_xml = bs_data.find('segment', {'id': seg_id})
            parent_id = segments[seg_id].parent_id
            
            if not parent_id:  # то есть, что это сегмент с предложением 
                segments[seg_id].sentence = segments[seg_id].seg_text
            else:

                search_area = segments[parent_id].available_search_area
                m = re.search('(?<![A-я])' + re.escape(segments[seg_id].seg_text) + '(?![A-я])', search_area)
                start, end = m.start(), m.end()
                segments[seg_id].char_start = segments[parent_id].char_start + segments[parent_id].prev_char_count + start
                segments[seg_id].char_end = segments[parent_id].char_start + segments[parent_id].prev_char_count + end
                segments[seg_id].sentence = segments[parent_id].sentence
                
                if segment_xml.get('features'):
                    segments[seg_id].features = segment_xml.get('features')
                if segment_xml.get('comment'):
                    segments[seg_id].govern_model = segment_xml.get('comment')

                segments[parent_id].available_search_area = segments[parent_id].available_search_area[end:]
                segments[parent_id].prev_char_count += m.end()
        return segments
            
    def extract_math_annotation(self):
        annot_segments = []
        segments = XML2Database.parse_annotation_segments_(self.bs_data)
        for seg_id in segments:
            if segments[seg_id].features:
                sentences = list(self.nlp(segments[seg_id].sentence).sents)
                sentences_starts = [s.start_char for s in sentences]
                segment_sent = sentences_starts.index([s for s in sentences_starts if s <= segments[seg_id].char_start][-1])
                new_char_start = segments[seg_id].char_start - sentences_starts[segment_sent]
                new_char_end = segments[seg_id].char_end - sentences_starts[segment_sent]
                annot_segments.append({
                    'sentence': sentences[segment_sent].text,
                    'segment_text': segments[seg_id].seg_text,
                    'char_start': new_char_start,
                    'char_end': new_char_end,
                    'annot': segments[seg_id].features,
                    'govern_model': segments[seg_id].govern_model
                })
                bolded_print(sentences[segment_sent].text, (new_char_start, new_char_end))
        self.db.add_math_annotation(annot_segments)
        print('all math annotation added to database')

In [15]:
xml_filename = 'Производная.Начало.xml'
db_path = 'math_corpus_database.db'

xml_to_db = XML2Database(xml_filename, db_path)
xml_to_db.extract_sentences()
# xml_to_db.extract_math_annotation()

207it [01:53,  1.82it/s]


OperationalError: near ".": syntax error

In [22]:
db = DBHandler(db_path)
db.change_text_status(2, 2)

In [23]:
del db
del xml_to_db